In [ ]:
%%capture
!pip3 install sentencepiece
!pip3 install transformers
!pip3 install imbalanced-learn
!pip3 install pytorch-metric-learning

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import torch
import os
import random
import time
import numpy as np
import pandas as pd
from typing import Tuple

from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from torch import nn, optim
from torch.utils.data import TensorDataset,RandomSampler,SequentialSampler
import torch.nn.functional as F
import torch.nn as nn

from pytorch_metric_learning import losses as loss_fun
from pytorch_metric_learning.distances import CosineSimilarity
from pytorch_metric_learning.reducers import ThresholdReducer
from pytorch_metric_learning.regularizers import LpRegularizer

from tensorflow.keras.preprocessing.sequence import pad_sequences

from transformers import BertForSequenceClassification, BertTokenizer, BertModel
from transformers import AdamW, get_linear_schedule_with_warmup

from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
from collections import defaultdict
from textwrap import wrap
from pylab import rcParams

# Seed
seed = 666
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
class RoBERTa_dataset(Dataset):
    def __init__(self, data, labels):
        super(RoBERTa_dataset, self).__init__()
        self.data = data
        self.labels = torch.tensor(labels)

    def __getitem__(self, idx):
        return self.labels[idx], self.data[idx]

    def __len__(self):
        return len(self.labels)

class ImdbDataset(Dataset):

    def __init__(self, reviews, targets, tokenizer, max_len):
        self.reviews = reviews
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, item):
        review = str(self.reviews[item])
        target = self.targets[item]

        encoding = self.tokenizer.encode_plus(
        review,
        add_special_tokens=True,
        max_length=self.max_len,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True
        )

        input_ids = pad_sequences(encoding['input_ids'], maxlen=self.max_len, dtype=torch.Tensor ,truncating="post",padding="post")
        input_ids = input_ids.astype(dtype = 'int64')
        input_ids = torch.tensor(input_ids)

        attention_mask = pad_sequences(encoding['attention_mask'], maxlen=self.max_len, dtype=torch.Tensor ,truncating="post",padding="post")
        attention_mask = attention_mask.astype(dtype = 'int64')
        attention_mask = torch.tensor(attention_mask)

        return {
        'review_text': review,
        'input_ids': input_ids,
        'attention_mask': attention_mask.flatten(),
        'targets': torch.tensor(target, dtype=torch.long)
        }

In [ ]:
class Sampler(object):
    """Base class for all Samplers.
    Every Sampler subclass has to provide an __iter__ method, providing a way
    to iterate over indices of dataset elements, and a __len__ method that
    returns the length of the returned iterators.
    """

    def __init__(self, data_source):
        pass

    def __iter__(self):
        raise NotImplementedError

    def __len__(self):
        raise NotImplementedError


class StratifiedSampler(Sampler):
    """Stratified Sampling
    Provides equal representation of target classes in each batch
    """
    def __init__(self, class_vector, batch_size):
        """
        Arguments
        ---------
        class_vector : torch tensor
            a vector of class labels
        batch_size : integer
            batch_size
        """
        self.n_splits = int(class_vector.size(0) / batch_size)
        self.class_vector = class_vector

    def gen_sample_array(self):
        try:
            from sklearn.model_selection import StratifiedShuffleSplit
        except:
            print('Need scikit-learn for this functionality')
        import numpy as np

        s = StratifiedShuffleSplit(n_splits=self.n_splits, test_size=0.5)
        X = torch.randn(self.class_vector.size(0),2).numpy()
        y = self.class_vector.numpy()
        s.get_n_splits(X, y)

        train_index, test_index = next(s.split(X, y))
        return np.hstack([train_index, test_index])

    def __iter__(self):
        return iter(self.gen_sample_array())

    def __len__(self):
        return len(self.class_vector)

In [ ]:
def getloader_undersample(X,y_digit,train_stratified,aspect,tokenizer,max_seq_lenght,max_size,drop_last=True):
    pos_train = []
    neg_train = []
    pos_test = []
    neg_test = []
    for data in train_stratified:
        if y_digit[data]==1:
            pos_train.append(X[data])
        else:
            neg_train.append(X[data])

    neg_train = random.sample(neg_train,len(pos_train))


    tartget_data = pos_train+neg_train
    tartget_label = [1]*len(pos_train)+[0]*len(neg_train)


    dataset = ImdbDataset(tartget_data,tartget_label,tokenizer,max_len=max_size)
    return DataLoader(dataset, batch_size=32, shuffle=True, num_workers=1, pin_memory=True, drop_last=drop_last)

def getloader(X,y_digit,train_stratified,aspect,tokenizer,max_seq_lenght,max_size,drop_last=True):
    pos_train = []
    neg_train = []
    pos_test = []
    neg_test = []
    for data in train_stratified:
        if y_digit[data]==1:
            pos_train.append(X[data])
        else:
            neg_train.append(X[data])

    tartget_data = pos_train+neg_train
    tartget_label = [1]*len(pos_train)+[0]*len(neg_train)


    dataset = ImdbDataset(tartget_data,tartget_label,tokenizer,max_len=max_size)
    return DataLoader(dataset, batch_size=32, shuffle=True, num_workers=1, pin_memory=True, drop_last=drop_last)


def get_train_valid_Bert_undersample(samplestrategy,X,y_digit,aspect,train_stratified,val_stratified,x2, tokenizer, max_seq_lenght):
    if samplestrategy == True:
        loader_train = getloader_undersample(X,y_digit,train_stratified,aspect,tokenizer,max_seq_lenght,max_size=max_seq_lenght)
    else:
        loader_train = getloader(X,y_digit,train_stratified,aspect,tokenizer,max_seq_lenght,max_size=max_seq_lenght)

    loader_test = getloader(X,y_digit,x2,aspect,tokenizer,max_seq_lenght,max_size=max_seq_lenght)

    loader_valid = getloader(X,y_digit,val_stratified,aspect,tokenizer,max_seq_lenght,max_size=max_seq_lenght)
    return loader_train, loader_valid, loader_test

def getloader_all(X,y_digit,tokenizer,max_size,drop_last=True):
    pos_train = []
    neg_train = []
    pos_test = []
    neg_test = []
    for data in range(len(y_digit)):
        print(data)
        if y_digit[data]==1:
            pos_train.append(X[data])
        else:
            neg_train.append(X[data])

    tartget_data = pos_train+neg_train
    tartget_label = [1]*len(pos_train)+[0]*len(neg_train)

    dataset = ImdbDataset(tartget_data,tartget_label,tokenizer,max_len=max_size)
    return DataLoader(dataset, batch_size=16, shuffle=True, num_workers=1, pin_memory=True, drop_last=drop_last)

In [ ]:
def writeoutput(info):
      mylog = open('record_tmp_2.log',mode = 'a', encoding='utf_8')
      print(info,file=mylog)
      mylog.close()

def translate(label,aspect):
      label_all = []
      count = 0

      for i in range(len(label)):
            if aspect in label[i]:
                  label_all.append(1)
                  count+=1
            else:
                  label_all.append(0)
      return label_all,count

#分层抽样   x1是数据的标号，x1里拆分出10%的验证集，ydigit是所有数据的label
def returnlabel(x1,y_digit):
      count=0
      count_negative = 0
      y_pos = []
      y_neg = []
      for x in x1:
            if y_digit[x]==1:
                  count+=1
                  y_pos.append(x)
            else:
                  count_negative+=1
                  y_neg.append(x)
      # ratio = float(count/(count+count_negative))
      # 1/10正例的个数，四舍五入
      pos_train,pos_val = train_test_split(y_pos,train_size=0.9)
      neg_train,neg_val = train_test_split(y_neg,train_size=0.9)


      return pos_train+neg_train,pos_val+neg_val

def aspect_wise_data(x1,y_digit,undersample):
    pos_train = []
    neg_train = []
    pos_test = []
    neg_test = []

    count = 0
    for data in x1:
      if y_digit[data] == 1:
        count = count+1

    if count < 100:
      for data in x1:
        if y_digit[data]==1:
            pos_train.append(data)
            pos_train.append(data)
        else:
            neg_train.append(data)
    else:
      for data in x1:
        if y_digit[data]==1:
            pos_train.append(data)
        else:
            neg_train.append(data)

    if undersample == True:
      neg_train = random.sample(neg_train,len(pos_train))
      btch_size=8
    else:
      pos_train = random.sample(pos_train, len(pos_train))
      btch_size=32

    tartget_data = pos_train+neg_train
    tartget_label = [1]*len(pos_train)+[0]*len(neg_train)

    sampler = StratifiedSampler(class_vector=torch.from_numpy(np.array(tartget_label)), batch_size=btch_size)
    dataset = ImdbDataset(tartget_data,tartget_label,tokenizer,max_len=160)
    return DataLoader(dataset, batch_size=btch_size, sampler=sampler, num_workers=1, pin_memory=True, drop_last=True)

In [ ]:
def supervisedContrastiveTraining(model, train_dataloader, epochs, device, optimizer, scheduler):

  #change here for using different loss function from https://kevinmusgrave.github.io/pytorch-metric-learning/
  loss_function = loss_fun.SupConLoss(temperature=0.1,embedding_regularizer = LpRegularizer())

  for epoch in range(epochs):
    losses = []
    model.zero_grad()
    model.train()

    for d in train_dataloader:
      input_ids = d["input_ids"].reshape(32, 160).to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask)

      hidden_states = outputs[0]

      supcon_fea_cls = F.normalize(hidden_states[:,0,:],dim=1)
      #supcon_fea_cls = F.normalize(hidden_states[-1][:,0,:],dim=1)

      loss = loss_function(supcon_fea_cls, targets)
      if not torch.isnan(loss):
        losses.append(loss.item())
        #print(loss)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
    print('Contrastive Loss Mean: ', np.mean(losses))

In [ ]:
def train_epoch(model, data_loader, optimizer, device, scheduler, n_examples):
  model.zero_grad()
  model = model.train()
  losses = []
  acc = 0
  counter = 0

  for d in data_loader:
        input_ids = d["input_ids"].reshape(32,160).to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)

        outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = targets)
        loss = outputs[0]
        logits = outputs[1]

        _, prediction = torch.max(outputs[1], dim=1)
        targets = targets.cpu().detach().numpy()
        prediction = prediction.cpu().detach().numpy()
        try:
              accuracy = metrics.accuracy_score(targets.tolist(), prediction.tolist())
        except:
              print(targets)
              print(prediction)
        acc += accuracy
        losses.append(loss.item())

        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        counter = counter + 1
  return acc / counter, np.mean(losses)

In [ ]:
def eval_model(model, data_loader, device, n_examples):
  model = model.eval()
  losses = []
  f1_sum = 0
  counter = 0
  y_pred = []
  y_true = []
  with torch.no_grad():
        for d in data_loader:
              input_ids = d["input_ids"].reshape(32,160).to(device)
              attention_mask = d["attention_mask"].to(device)
              targets = d["targets"].to(device)

              outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = targets)
              loss = outputs[0]
              logits = outputs[1]

              _, prediction = torch.max(outputs[1], dim=1)
              targets = targets.cpu().detach().numpy()
              prediction = prediction.cpu().detach().numpy()
              f1 = metrics.f1_score(targets, prediction)

              f1_sum += f1
              y_pred.extend(prediction)
              y_true.extend(targets.tolist())
              losses.append(loss.item())
              counter += 1

  return metrics.f1_score(y_true, y_pred,average='weighted'), np.mean(losses), classification_report(y_true, y_pred, labels=[0,1], digits=4),metrics.precision_score(y_true, y_pred,average='weighted'),metrics.recall_score(y_true, y_pred,average='weighted'), metrics.matthews_corrcoef(y_true, y_pred), metrics.roc_auc_score(y_true, y_pred)


In [ ]:
PRE_TRAINED_MODEL_NAME = "jeniya/BERTOverflow"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
#data load
datapath = '/content/BenchmarkUddinSO-ConsoliatedAspectSentiment.csv'
data = pd.read_csv(datapath,sep=',')
X = data['sent'] # data
y = data['codes'] # label

k = 10 # 10 fold cross validation
kf = StratifiedKFold(n_splits=k, random_state=None)

In [ ]:
# aspect_list = ['Performance','Usability','Security','Community','Compatibility','Portability','Documentation','Bug','Legal','OnlySentiment','Other']

aspects = ['Performance']
EPOCHS = 5
BATCH_SIZE = 32
learn_rate_list = [5e-5]
undersample = [False]

In [ ]:
def contrastive_training(X, con_y_digit):
  print("Contrastive Learning:")

  model = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME, output_hidden_states=True)
  model = model.to(device)
  param_optimizer = list(model.named_parameters())
  no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay':0.0}]
  optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=learn_rate_list[0])

  con_train = aspect_wise_data(X, con_y_digit, False)
  total_steps = len(con_train) * EPOCHS
  scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps=total_steps)

  supervisedContrastiveTraining(model, con_train, EPOCHS, device, optimizer, scheduler)
  return model

In [ ]:
def Aspect_Detection(aspect, learn_rate):

  dataframe = pd.DataFrame()
  avg_df = pd.DataFrame()

  best_pre=[]
  best_re=[]
  best_f=[]
  best_mcc=[]
  best_auc=[]

  count = 0

  y_digit, true_label_number = translate(y, aspect)
  print("True Label Number: ", true_label_number)

  for x1,x2 in kf.split(X,y_digit):
    print("inside k-fold iteration: ", count)
    print(len(x1))
    print(len(x2))

    train_stratified, val_stratified = returnlabel(x1, y_digit)
    best_F1 = -1
    train_iter, val_iter, test_iter = get_train_valid_Bert_undersample(False, X, y_digit, aspect, train_stratified, val_stratified, x2, tokenizer, max_seq_lenght=160)
    print("training data size: ", len(train_iter))
    print("testing data size: ", len(test_iter))

    '''uncomment these when fine-tuning with contrastive learning'''
    # model = contrastive_training(train_stratified, y_digit)
    # model.save_pretrained('/content/contrastive/')

    best_epoch_F1 = -1

    try:
      fine_model =  BertForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME, num_labels = 2) #replace PRE_TRAINED_MODEL_NAME with /content/contrastive/ for CL
    except:
      time.sleep(15)
      fine_model =  BertForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME, num_labels = 2) #replace PRE_TRAINED_MODEL_NAME with /content/contrastive/ for CL

    fine_model.to(device)
    fine_param_optimizer = list(fine_model.named_parameters())
    fine_no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    fine_optimizer_grouped_parameters = [
      {'params': [p for n, p in fine_param_optimizer if not any(nd in n for nd in fine_no_decay)], 'weight_decay': 0.01},
      {'params': [p for n, p in fine_param_optimizer if any(nd in n for nd in fine_no_decay)], 'weight_decay':0.0}]
    fine_optimizer = torch.optim.AdamW(fine_optimizer_grouped_parameters, lr=learn_rate)
    fine_total_steps = len(train_iter) * EPOCHS
    fine_scheduler = get_linear_schedule_with_warmup(fine_optimizer,num_warmup_steps=0,num_training_steps=fine_total_steps)

    for epoch in range(EPOCHS):
      print(f'Epoch {epoch + 1}/{EPOCHS}')
      print('-' * 10)

      #fine tuning
      print("Fine tuning")
      train_acc, train_loss = train_epoch(
            fine_model,
            train_iter,
            fine_optimizer,
            device,
            fine_scheduler,
            len(train_iter)
      )
      print(f'Train loss {train_loss} Train accuracy {train_acc}')
      val_F_1, val_loss, metrics_all, val_precision, val_recall, _, _ = eval_model(
            fine_model,
            val_iter,
            device,
            len(val_iter)
      )
      print(f'Val loss {val_loss} Val F-1 {val_F_1}')
      if val_F_1 > best_epoch_F1:
            best_epoch_F1 = val_F_1
            fine_model.save_pretrained('/content/best_model/')

    print("Epoch End--->")
    best_model = BertForSequenceClassification.from_pretrained('/content/best_model/', num_labels = 2)
    best_model.to(device)
    test_F_1, test_loss, metrics_all, test_precision, test_recall, test_mcc, test_auc = eval_model(
                best_model,
                test_iter,
                device,
                len(test_iter)
          )
    if test_F_1 > best_F1:
          best_F1 = test_F_1
          best_metrics = metrics_all
          best_precision = test_precision
          best_recall = test_recall
          best_learn_rate = learn_rate
          best_t_mcc = test_mcc
          best_t_auc = test_auc
          best_model.save_pretrained('/content/test_model/')

    #del model
    del fine_model
    del best_model

    best_pre.append(best_precision)
    best_re.append(best_recall)
    best_f.append(best_F1)
    best_mcc.append(best_t_mcc)
    best_auc.append(best_t_auc)

    print(best_metrics)
    print("MSCC: ", best_t_mcc, " AUC: ",best_t_auc)
    count += 1

  dataframe["aspect"] = [aspect]*10
  dataframe["learn_rate"] = [best_learn_rate]*10
  dataframe['sample strategy'] = [False]*10
  dataframe['best_precision'] = best_pre
  dataframe['best recall']=best_re
  dataframe['best F-1']=best_f
  dataframe['best MCC']=best_mcc
  dataframe['best AUC']=best_auc

  avg_df['aspect'] = [aspect]
  avg_df['avg_precision'] = [np.mean(best_pre)]
  avg_df['avg_recall'] = [np.mean(best_re)]
  avg_df['avg_f1'] = [np.mean(best_f)]
  avg_df['avg_mcc'] = [np.mean(best_mcc)]
  avg_df['avg_auc'] = [np.mean(best_auc)]

  print(dataframe.head())

  dataframe.to_csv('Detail result.csv', index=False)
  avg_df.to_csv('Average result.csv', index=False)

In [ ]:
Aspect_Detection(aspects[0], learn_rate_list[0])